<br>

# Introdução


In [ ]:
#!pip3 install traquitanas --upgrade


In [ ]:
import os
import re
import time
import unicodedata
from pathlib import Path

import my_driver
import pandas as pd
import requests
from paths import adds_path, driver_path, logs_path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from traquitanas.scrapping import adds, gecko

In [ ]:
download_path = Path('.').parent.absolute()

In [ ]:
import json
import shutil
import sys
import zipfile

from bs4 import BeautifulSoup
from paths import *
from selenium.webdriver import ActionChains, DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

<br>

# Drivers


In [ ]:
driver = my_driver.Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,
    my_download_path=download_path,
    verify_ssl=True
)
driver.add_extension_xpath(adds_path)

<br>

# Portal AgroFit


In [ ]:
# Portal
url = 'http://agrofit.agricultura.gov.br/agrofit_cons/principal_agrofit_cons'
driver.get(url)


<br>

# Culturas


In [ ]:
def get_ia_url():
    # Consulta de Ingredientes Ativos
    url = Path('http://agrofit.agricultura.gov.br/agrofit_cons').joinpath(
        '!ap_ing_ativo_consulta_cons'
    )
    driver.get(url.as_posix())


In [ ]:
def get_list_culturas():
    # Get URL
    get_ia_url()

    # Listar Culturas
    list_culturas = []
    culturas = driver.find_elements(
        By.XPATH, "//*[@name='p_id_cultura']//option"
    )
    for cultura in culturas:
        if cultura.text != 'Selecione':
            list_culturas.append(cultura.text)

    # Results
    list_culturas.sort()
    print(f'O AgroFit tem {len(list_culturas)} culturas agrícolas distintas')
    return list_culturas


In [ ]:
list_culturas = get_list_culturas()
list_culturas


In [ ]:
def set_cultura(cultura):
    # Get URL
    get_ia_url()

    # Set Cultura
    driver.find_element(
        By.XPATH, f"//*[@name='p_id_cultura']//option[text()='{cultura}']"
    ).click()
    driver.find_element(By.XPATH, "//input[@name='ConsultarDados']").click()


In [ ]:
cultura = 'Cana-de-açúcar'
set_cultura(cultura)


<br>

# Listar Ingredientes Ativos


In [ ]:
def get_ia_dataframe(driver):
    #
    tbody = driver.find_element(
        By.XPATH, '//*[@class="LabelCampo"]//*[@class="P"]'
    )

    #
    df_rows = []
    rows = tbody.find_elements(By.XPATH, './/tr')
    for row in rows:
        df_row = []
        # Header
        headers = row.find_elements(By.XPATH, './/td[@class="tituloinicial"]')
        for head in headers:
            df_row.append(head.text)
            df_row.append(f'{head.text} - Url')

        # Rows
        row_texts = row.find_elements(By.XPATH, './/td/*')
        for row_text in row_texts:
            df_row.append(row_text.text)
            df_row.append(row_text.get_attribute('href'))
        df_rows.append(df_row)

    # Create and Adjust Dataframe
    df = pd.DataFrame(df_rows)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df


In [ ]:
# Clica em Anterior
# driver.find_element_by_xpath("//input[@name='anterior']").click()

# Clica em Próximo até não mais existir
# driver.find_element_by_xpath("//input[@name='proximo']").click()


In [ ]:
def get_ia_dataframes():
    # Loop Over Dataframes
    dfs = []
    start = True
    while start:
        df = get_ia_dataframe(driver)
        dfs.append(df)

        # Clica em Próximo até não mais existir:
        try:
            driver.find_element(By.XPATH, "//input[@name='proximo']").click()
        except:
            start = False
        time.sleep(5)

    # Merge Dataframes
    df = pd.concat(dfs)
    df.head()

    # Avalia se as colunas são idênticas e, se for, deleta
    if df['Nome Comum - Url'].equals(df['Grupo Químico - Url']) and df[
        'Nome Comum - Url'
    ].equals(df['Classe(s) - Url']):
        df.drop(
            columns=['Nome Comum - Url', 'Grupo Químico - Url'], inplace=True
        )
        df.rename(columns={'Classe(s) - Url': 'URL'}, inplace=True)

        #
        df['id_ingrediente_ativo'] = df['URL'].apply(
            lambda x: x.split('p_id_ingrediente_ativo=')[-1]
        )
    else:
        print('Avaliar!')

    return df


In [23]:
# Results
df = get_ia_dataframes()
df


<br>

# Detalhes Ingrediente Ativo


In [ ]:
def get_ia_infos_dados_gerais(id_ingrediente_ativo):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        f'!ap_ing_ativo_detalhe_cons?p_id_ingrediente_ativo={id_ingrediente_ativo}',
    )
    driver.get(url)

    # Vai na Aba Dados Gerais
    driver.find_element(By.XPATH, "//*[@id='orelha1']").click()

    # Informações
    dict_infos = {
        # Infos
        'Nome Comum': driver.find_element(
            By.XPATH, "//*[@name='p_nm_comum']"
        ).get_attribute('value'),
        'Nome Comum': driver.find_element(
            By.XPATH, "//*[@name='p_nm_comum_portugues']"
        ).get_attribute('value'),
        'Número CAS': driver.find_element(
            By.XPATH, "//*[@name='p_nr_cas']"
        ).get_attribute('value'),
        'Número ANVISA': driver.find_element(
            By.XPATH, "//*[@name='p_nr_anvisa']"
        ).get_attribute('value'),
        # Química
        'Fórmula Bruta': driver.find_element(
            By.XPATH, "//*[@name='p_tx_formula_bruta']"
        ).get_attribute('value'),
        'Grupo Químico': driver.find_element(
            By.XPATH, "//*[@name='p_nm_grupo_quimico']"
        ).get_attribute('value'),
        'Nome Químico IUPAC': driver.find_element(
            By.XPATH, "//*[@name='p_nm_quimico_iupac']"
        ).get_attribute('value'),
        'Nome Químico CA': driver.find_element(
            By.XPATH, "//*[@name='p_nm_quimico_ca']"
        ).get_attribute('value'),
        # Classe
        'Classe / Categoria Agronômica': driver.find_element(
            By.XPATH, "//*[@name='p_ds_classe']"
        ).get_attribute('value'),
    }

    # Results
    return pd.DataFrame([dict_infos], columns=dict_infos.keys())


In [ ]:
def get_ia_infos_produtos_formulados(id_ingrediente_ativo):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_ing_ativo_detalhe_cons?p_id_ingrediente_ativo={}'.format(
            id_ingrediente_ativo
        ),
    )
    driver.get(url)

    # Vai na Aba Produtos Formulados
    driver.find_element(By.XPATH, "//*[@id='orelha2']").click()

    # Table
    tbody = driver.find_element(By.XPATH, 
        '//*[@style="overflow-x:scroll;overflow-y:scroll; width:680; height:230"]//*[@class="bordaTabelaContexto"]'
    )

    # Loop
    df_rows = []
    rows = tbody.find_elements(By.XPATH, './/tr')

    for row in rows:
        df_row = []
        # Header
        headers = row.find_elements(By.XPATH, './/td[@class="tituloinicial"]')
        for head in headers:
            df_row.append(head.text.strip())
            df_row.append('{} - Url'.format(head.text.strip()))

        # Rows
        row_texts = row.find_elements(By.XPATH, './/td/*')
        for row_text in row_texts:
            df_row.append(row_text.text.strip())
            df_row.append(row_text.get_attribute('href'))
        df_rows.append(df_row)

    # Create and Adjust Dataframe
    df = pd.DataFrame(df_rows)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.rename(columns={'Classe': 'Classe Tóx'}, inplace=True)
    df.rename(columns={df.columns[-2]: 'Classe Amb'}, inplace=True)
    df.drop(df.index[0], inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.head()

    # Avalia se as colunas são idênticas e, se for, deleta
    if (
        df['Produto - Url'].equals(df['Ingrediente Ativo(Grupo Químico) - Url'])
        and df['Produto - Url'].equals(df['Titular de Registro - Url'])
        and df['Produto - Url'].equals(df['Formulação - Url'])
        and df['Produto - Url'].equals(df['Classe - Url'])
    ):

        df.drop(
            columns=[
                'Produto - Url',
                'Ingrediente Ativo(Grupo Químico) - Url',
                'Titular de Registro - Url',
                'Formulação - Url',
                'Classe - Url',
            ],
            inplace=True,
        )

        df.columns.values[-1] = 'URL'
        df['URL'] = df['URL'].apply(
            lambda x: os.path.join(
                'http://agrofit.agricultura.gov.br/agrofit_cons',
                x.split('&p_tipo_janela')[0].split("javascript: NewWindow('")[
                    -1
                ],
            )
        )
        df['id_produto_formulado_tecnico'] = df['URL'].apply(
            lambda x: x.split('id_produto_formulado_tecnico=')[-1]
        )
    else:
        print('Avaliar!')

    # Results
    return df


In [ ]:
id_ingrediente_ativo = 97


In [ ]:
get_ia_infos_dados_gerais(id_ingrediente_ativo)


In [ ]:
get_ia_infos_produtos_formulados(id_ingrediente_ativo)


<br>

# Produtos Formulados


In [ ]:
id_produto_formulado_tecnico = 13070


<br>

## Dados Gerais


In [ ]:
def get_pf_infos_dados_gerais(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Dados Gerais
    driver.find_element_by_xpath("//*[@id='orelha1']").click()

    # Informações
    dict_infos = {
        # Identificação
        'Marca Comercial': driver.find_element(By.XPATH, 
            "//*[@name='p_nm_marca_comercial']"
        ).get_attribute('value'),
        'Título do Registro': driver.find_element(By.XPATH, 
            "//*[@name='p_nm_registrante_empresa']"
        ).get_attribute('value'),
        'Número do Registro': driver.find_element(By.XPATH, 
            "//*[@name='p_nr_registro']"
        ).get_attribute('value'),
        'CNPJ': driver.find_element(By.XPATH, 
            "//*[@name='p_nr_cnpj']"
        ).get_attribute('value'),
        # Classificação
        'Classe / Categoria Agronômica': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_classe']"
        ).get_attribute('value'),
        'Classificação Toxicólogica': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_classificacao_tox']"
        ).get_attribute('value'),
        'Classificação Ambiental': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_classificacao_amb']"
        ).get_attribute('value'),
        # Aplicação
        'Modo de Ação': driver.find_element(By.XPATH, 
            "//*[@name='p_tx_modo_acao']"
        ).get_attribute('value'),
        'Técnica de Aplicação': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_tecnica_aplicacao']"
        ).get_attribute('value'),
        'Compatibilidade': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_compatibilidade']"
        ).get_attribute('value'),
        'Inflamável': driver.find_element(By.XPATH, 
            "//*[contains(text(), '{}')]/following-sibling::td".format(
                'Inflamável'
            )
        ).text,
        'Corrosivo': driver.find_element(By.XPATH, 
            "//*[contains(text(), '{}')]/following-sibling::td".format(
                'Corrosivo'
            )
        ).text,
        # Outros
        'Embalagem do Produto': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_empalagem']"
        ).get_attribute('value'),
        'Observação': driver.find_element(By.XPATH, 
            "//*[@name='p_tx_observacao']"
        ).get_attribute('value'),
    }

    return pd.DataFrame([dict_infos], columns=dict_infos.keys())


In [ ]:
get_pf_infos_dados_gerais(id_produto_formulado_tecnico)


<br>

## Composição


In [ ]:
def get_pf_infos_composicao_part_1(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Composição
    driver.find_element_by_xpath("//*[@id='orelha2']").click()

    # Informações
    dict_infos = {
        # Identificação
        'Formulação': driver.find_element(By.XPATH, 
            "//*[@name='p_ds_formulacao']"
        ).get_attribute('value'),
        'Quantidade Ingrediente Inerte': driver.find_element(By.XPATH, 
            "//*[@name='p_qt_ingrediente_inerte']"
        ).get_attribute('value'),
        'Percentual de Inerte': driver.find_element(By.XPATH, 
            "//*[@name='p_nr_percentual_inerte']"
        ).get_attribute('value'),
        'Unidade de Medida': driver.find_element(By.XPATH, 
            "//*[@name='p_nm_unidade_medida_inerte']"
        ).get_attribute('value'),
    }

    return pd.DataFrame([dict_infos], columns=dict_infos.keys())


In [ ]:
def get_pf_infos_composicao_part_2(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Composição
    driver.find_element(By.XPATH, "//*[@id='orelha2']").click()

    # Table
    tbody = driver.find_element(By.XPATH, '//*[@id="ingredientes"]')
    # //*[@class="bordaTabelaContexto"]

    # Loop
    df_rows = []
    rows = tbody.find_elements(By.XPATH, './/tr')

    for row in rows:
        df_row = []
        # Header
        headers = row.find_elements(By.XPATH, './/td[@class="tituloinicial"]')
        for head in headers:
            df_row.append(head.text.strip())

        # Rows
        row_texts = row.find_elements(By.XPATH, './/td/*')
        for row_text in row_texts:
            df_row.append(row_text.get_attribute('value'))
        df_rows.append(df_row)

    df_rows[0].pop()
    df_rows[0].pop()
    df_rows[0].append('Nome Químico IUPAC')
    df_rows[0].append('Nome Químico CA')
    df_rows[0].append('Fórmula Bruta')
    df_rows.remove(['IUPAC', 'CA'])

    # Create and Adjust Dataframe
    df = pd.DataFrame(df_rows)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df


In [ ]:
get_pf_infos_composicao_part_1(id_produto_formulado_tecnico)


In [ ]:
get_pf_infos_composicao_part_2(id_produto_formulado_tecnico)


<br>

## Indicações


In [ ]:
def get_pf_infos_indicacao_uso_dose(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Indicações de Uso/Doses
    driver.find_element(By.XPATH, "//*[@id='orelha3']").click()

    # Table
    tbody = driver.find_element(By.XPATH, '//*[@id="doses"]')

    # Loop
    df_rows = []
    rows = tbody.find_elements(By.XPATH, './/tr')

    for row in rows:
        df_row = []
        # Header
        headers = row.find_elements(By.XPATH, './/td[@class="tituloinicial"]')
        for head in headers:
            df_row.append(head.text.strip())

        # Rows
        row_texts = row.find_elements(By.XPATH, 
            './/td[@class="bordaTabelaContexto"]'
        )
        for row_text in row_texts:
            df_row.append(row_text.text.strip())
        df_rows.append(df_row)

    list_columns = [
        '{}'.format(df_rows[0][0]),
        '{} - {}'.format(df_rows[0][1], df_rows[1][0]),
        '{} - {}'.format(df_rows[0][1], df_rows[1][1]),
        '{} - {}'.format(df_rows[0][2], df_rows[1][2]),
        '{} - {}'.format(df_rows[0][2], df_rows[1][3]),
        '{} - {}'.format(df_rows[0][3], df_rows[1][4]),
        '{} - {}'.format(df_rows[0][3], df_rows[1][5]),
        '{} - {}'.format(df_rows[0][3], df_rows[1][6]),
        '{} - {}'.format(df_rows[0][3], df_rows[1][7]),
        '{} - {}'.format(df_rows[0][4], df_rows[1][8]),
        '{} - {}'.format(df_rows[0][4], df_rows[1][9]),
        '{}'.format(df_rows[0][5]),
    ]

    df_rows_temp = []
    df_rows_temp.append(list_columns)
    df_rows = df_rows[2:]
    for df_row in df_rows:
        df_row.insert(3, df_row[-1])
        df_row = df_row[0:12]
        df_rows_temp.append(df_row)

    # Create and Adjust Dataframe
    df = pd.DataFrame(df_rows_temp)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df


In [ ]:
get_pf_infos_indicacao_uso_dose(id_produto_formulado_tecnico)


<br>

## Documentos


In [ ]:
def get_doc_table(instituion):
    # Dictionary
    dict_instituion = {
        'mma': 'N8',
        'anvisa': 'N9',
        'ibama': 'N10',
        'bula': 'N7',
    }

    # Table
    tbody = driver.find_element(By.XPATH, 
        '//*[@id="{}"]//*[@id="documentosMAPA"]'.format(
            dict_instituion[instituion]
        )
    )

    # Loop
    df_rows = []
    rows = tbody.find_elements(By.XPATH, './/tr')
    for row in rows:
        # Rows
        df_row = []
        row_texts = row.find_elements(By.XPATH, './/td/*')
        for row_text in row_texts:
            df_row.append(row_text.text.strip())
            df_row.append(row_text.get_attribute('href'))
        df_rows.append(df_row)

    list_columns = [
        'Certificado',
        'Certificado - Url',
        'Tipo de Documento',
        'Tipo de Documento - Url',
        'Data Inclusão',
        'Data Inclusão - Url',
    ]

    # Create Dataframe
    df = pd.DataFrame(df_rows[2:], columns=list_columns)

    # Avalia se as colunas são idênticas e, se for, deleta
    if df['Certificado - Url'].equals(df['Tipo de Documento - Url']) and df[
        'Certificado - Url'
    ].equals(df['Data Inclusão - Url']):

        df.drop(
            columns=[
                'Certificado - Url',
                'Tipo de Documento - Url',
            ],
            inplace=True,
        )

        df.columns.values[-1] = 'URL'
        # df['URL'] = df['URL'].apply(lambda x: os.path.join('http://agrofit.agricultura.gov.br/agrofit_cons', x.split('&p_tipo_janela')[0].split("javascript: NewWindow('")[-1]))
        # df['id_produto_formulado_tecnico'] = df['URL'].apply(lambda x: x.split('id_produto_formulado_tecnico=')[-1])
        
    else:
        print('Avaliar!')
    return df


In [ ]:
def download_files(df, input_path):
    for i, row in df.iterrows():
        #
        url = row['URL']
        filename = requests.utils.unquote(os.path.basename(url))
        filename = (
            unicodedata.normalize('NFKD', filename)
            .encode('ascii', 'ignore')
            .decode('utf8')
        )
        filename = re.sub('[!@#$;,]', '', filename)
        filename = filename.strip()
        print(filename)

        # Download File
        r = requests.get(url)
        with open(os.path.join(input_path, filename), 'wb') as f:
            f.write(r.content)


<br>

### MMA


In [ ]:
def get_pf_infos_docs_mma(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        f'!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={id_produto_formulado_tecnico}',
    )
    driver.get(url)

    # Aba Documentos
    driver.find_element(By.XPATH, "//*[@id='orelha6']").click()

    # Aba Documentos MMA
    driver.find_element(By.XPATH, "//*[@id='orelha8']").click()

    return get_doc_table('mma')


In [ ]:
df = get_pf_infos_docs_mma(id_produto_formulado_tecnico)
df


<br>

### ANVISA


In [ ]:
def get_pf_infos_docs_anvisa(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Documentos
    driver.find_element(By.XPATH, "//*[@id='orelha6']").click()

    # Aba Documentos ANVISA
    driver.find_element(By.XPATH, "//*[@id='orelha9']").click()

    return get_doc_table('anvisa')


In [ ]:
df = get_pf_infos_docs_anvisa(id_produto_formulado_tecnico)
df


<br>

### IBAMA


In [ ]:
def get_pf_infos_docs_ibama(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={id_produto_formulado_tecnico}',
    )
    driver.get(url)

    # Aba Documentos
    driver.find_element(By.XPATH, "//*[@id='orelha6']").click()

    # Aba Documentos IBAMA
    driver.find_element(By.XPATH, "//*[@id='orelha10']").click()

    return get_doc_table('ibama')


In [ ]:
get_pf_infos_docs_ibama(id_produto_formulado_tecnico)


<br>

## Bula, Rótulos e Certificado


In [ ]:
def get_pf_infos_bula(id_produto_formulado_tecnico):
    # URL
    url = os.path.join(
        'http://agrofit.agricultura.gov.br/agrofit_cons',
        '!ap_produto_form_detalhe_cons?p_id_produto_formulado_tecnico={}'.format(
            id_produto_formulado_tecnico
        ),
    )
    driver.get(url)

    # Aba Bula, Rótulo e Certificado
    driver.find_element(By.XPATH, "//*[@id='orelha7']").click()

    return get_doc_table('bula')


In [ ]:
df = get_pf_infos_bula(id_produto_formulado_tecnico)
df


<br>

# Loop


<br>

## Ingredientes Ativos


In [ ]:
# Get Culturas
list_culturas = get_list_culturas()
# list_culturas = list_culturas[18:22]
# list_culturas


In [ ]:
# Lists
dfs_ia = []

# loop
for cultura in list_culturas:
    print(cultura)

    try:
        # Set Cultura
        set_cultura(cultura)
        time.sleep(1)

        # Ingrediente Ativo
        df_ia = get_ia_dataframes()
        df_ia['Cultura'] = cultura
        dfs_ia.append(df_ia)
    except Exception as e:
        print(e)

# Results
dfs_ia = pd.concat(dfs_ia)

# Save
input_path = os.path.join('..', 'data')
os.makedirs(input_path, exist_ok=True)
dfs_ia.to_csv(
    os.path.join(input_path, 'tab_ia_culturas.csv'),
    index=False,
)


<br>

## Ingredientes Ativos - Detalhes


In [ ]:
# Read
input_path = os.path.join('..', 'data')
dfs_ia = pd.read_csv(
    os.path.join(input_path, 'tab_ia_culturas.csv'),
)

# List
list_ia = list(set(list(dfs_ia['id_ingrediente_ativo'])))
list_ia.sort()
print(len(list_ia))


In [ ]:
# Lists
dfs_ia_infos_dados_gerais = []
dfs_ia_infos_produtos_formulados = []

# Loop
for ia in list_ia:
    print(ia)

    # Get Data
    df_ia_infos_dados_gerais = get_ia_infos_dados_gerais(ia)
    df_ia_infos_dados_gerais['ia'] = ia
    time.sleep(2)

    df_ia_infos_produtos_formulados = get_ia_infos_produtos_formulados(ia)
    df_ia_infos_produtos_formulados['ia'] = ia
    time.sleep(2)

    # Append
    dfs_ia_infos_dados_gerais.append(df_ia_infos_dados_gerais)
    dfs_ia_infos_produtos_formulados.append(df_ia_infos_produtos_formulados)

# Results
dfs_ia_infos_dados_gerais = pd.concat(dfs_ia_infos_dados_gerais)
dfs_ia_infos_produtos_formulados = pd.concat(dfs_ia_infos_produtos_formulados)

# Save
dfs_ia_infos_dados_gerais.to_csv(
    input_path / 'tab_ia_dados_gerais.csv',
    index=False,
)
dfs_ia_infos_produtos_formulados.to_csv(
    input_path / 'tab_ia_produtos_formulados.csv',
    index=False,
)


<br>

## Produtos Formulados - Básicos


In [ ]:
# Read

dfs_ia_infos_produtos_formulados = pd.read_csv(
    input_path / 'tab_ia_produtos_formulados.csv',
)

# List
list_pf = list(
    set(list(dfs_ia_infos_produtos_formulados['id_produto_formulado_tecnico']))
)
list_pf.sort()
print(len(list_pf))


In [ ]:
# Lists
dfs_pf_infos_dados_gerais = []
dfs_pf_infos_composicao_part_1 = []
dfs_pf_infos_composicao_part_2 = []
dfs_pf_infos_indicacao_uso_dose = []

# Loop
for pf in list_pf:
    print(pf)

    # Get Data
    df_pf_infos_dados_gerais = get_pf_infos_dados_gerais(pf)
    df_pf_infos_dados_gerais['pf'] = pf
    time.sleep(2)

    df_pf_infos_composicao_part_1 = get_pf_infos_composicao_part_1(pf)
    df_pf_infos_composicao_part_1['pf'] = pf
    time.sleep(2)

    df_pf_infos_composicao_part_2 = get_pf_infos_composicao_part_2(pf)
    df_pf_infos_composicao_part_2['pf'] = pf
    time.sleep(2)

    df_pf_infos_indicacao_uso_dose = get_pf_infos_indicacao_uso_dose(pf)
    df_pf_infos_indicacao_uso_dose['pf'] = pf
    time.sleep(2)

    # Append
    dfs_pf_infos_dados_gerais.append(df_pf_infos_dados_gerais)
    dfs_pf_infos_composicao_part_1.append(df_pf_infos_composicao_part_1)
    dfs_pf_infos_composicao_part_2.append(df_pf_infos_composicao_part_2)
    dfs_pf_infos_indicacao_uso_dose.append(df_pf_infos_indicacao_uso_dose)

# Results
dfs_pf_infos_dados_gerais = pd.concat(dfs_pf_infos_dados_gerais)
dfs_pf_infos_composicao_part_1 = pd.concat(dfs_pf_infos_composicao_part_1)
dfs_pf_infos_composicao_part_2 = pd.concat(dfs_pf_infos_composicao_part_2)
dfs_pf_infos_indicacao_uso_dose = pd.concat(dfs_pf_infos_indicacao_uso_dose)

# Save
dfs_pf_infos_dados_gerais.to_csv(
    input_path / 'tab_pf_dados_gerais.csv',
    index=False,
)
dfs_pf_infos_composicao_part_1.to_csv(
    input_path / 'tab_pf_composicao_part_1.csv',
    index=False,
)
dfs_pf_infos_composicao_part_2.to_csv(
    input_path / 'tab_pf_composicao_part_2.csv',
    index=False,
)
dfs_pf_infos_indicacao_uso_dose.to_csv(
    input_path / 'tab_pf_indicacao_uso_dose.csv',
    index=False,
)


<br>

## Produtos Formulados - Documentos


In [ ]:
# Lists
dfs_pf_infos_docs_mma = []
dfs_pf_infos_docs_anvisa = []
dfs_pf_infos_docs_ibama = []
dfs_pf_infos_bula = []

# Loop
for pf in list_pf:
    print(pf)

    # Get Data
    df_pf_infos_docs_mma = get_pf_infos_docs_mma(pf)
    df_pf_infos_docs_mma['pf'] = pf
    time.sleep(2)

    df_pf_infos_docs_anvisa = get_pf_infos_docs_anvisa(pf)
    df_pf_infos_docs_anvisa['pf'] = pf
    time.sleep(2)

    df_pf_infos_docs_ibama = get_pf_infos_docs_ibama(pf)
    df_pf_infos_docs_ibama['pf'] = pf
    time.sleep(2)

    dfs_pf_infos_bula = get_pf_infos_bula(pf)
    dfs_pf_infos_bula['pf'] = pf
    time.sleep(2)

    # Download
    input_path = os.path.join('..', 'data', pf, 'mma')
    os.makedirs(input_path, exist_ok=True)
    download_files(df_pf_infos_docs_mma, input_path)

    input_path = os.path.join('..', 'data', pf, 'anvisa')
    os.makedirs(input_path, exist_ok=True)
    download_files(df_pf_infos_docs_anvisa, input_path)

    input_path = os.path.join('..', 'data', pf, 'ibama')
    os.makedirs(input_path, exist_ok=True)
    download_files(df_pf_infos_docs_ibama, input_path)

    input_path = os.path.join('..', 'data', pf, 'bula')
    os.makedirs(input_path, exist_ok=True)
    download_files(df_pf_infos_bula, input_path)

    # Append
    dfs_pf_infos_docs_mma.append(df_pf_infos_docs_mma)
    dfs_pf_infos_docs_anvisa.append(df_pf_infos_docs_anvisa)
    dfs_pf_infos_docs_ibama.append(df_pf_infos_docs_ibama)
    dfs_pf_infos_bula.append(df_pf_infos_bula)

# Results
dfs_pf_infos_docs_mma = pd.concat(dfs_pf_infos_docs_mma)
dfs_pf_infos_docs_anvisa = pd.concat(dfs_pf_infos_docs_anvisa)
dfs_pf_infos_docs_ibama = pd.concat(dfs_pf_infos_docs_ibama)
dfs_pf_infos_bula = pd.concat(dfs_pf_infos_bula)

# Save
dfs_pf_infos_docs_mma.to_csv(
    input_path / 'tab_pf_docs_mma.csv',
    index=False,
)
dfs_pf_infos_docs_anvisa.to_csv(
    input_path / 'tab_pf_docs_anvisa.csv',
    index=False,
)
dfs_pf_infos_docs_ibama.to_csv(
    input_path / 'tab_pf_docs_ibama.csv',
    index=False,
)
dfs_pf_infos_bula.to_csv(
    input_path / 'tab_pf_docs_bula.csv',
    index=False,
)


In [ ]:
driver.quit()
